In [18]:
import cv2
import numpy as np

import os
import time
import math

from utility.h5data import h5DataWrite

In [19]:
DATA_FORMAT = ".png"
DATAPATH_TRAIN = "./imageTrain/"
DATAPATH_VALIDATE = "./imageValidate/"

SIZE_X = 160
SIZE_Y = 120

SCALE = 4
INTERPOLATION = cv2.INTER_LINEAR

FILENAME_TRAIN = "yayoi_waifu2x_dataTrain_" + str(SCALE)  + ".h5"
FILENAME_VALIDATE = "yayoi_waifu2x_dataValidate_" + str(SCALE) + ".h5"

### OpenCV interpolation methods
INTER_NEAREST - a nearest-neighbor interpolation<br>
INTER_LINEAR - a bilinear interpolation (used by default)<br>
INTER_AREA - resampling using pixel area relation. It may be a preferred method for image decimation, as it gives moire’-free results. But when the image is zoomed, it is similar to the INTER_NEAREST method.<br>
INTER_CUBIC - a bicubic interpolation over 4x4 pixel neighborhood<br>
INTER_LANCZOS4 - a Lanczos interpolation over 8x8 pixel neighborhood<br>

In [20]:
def parseData(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = []
    label = []

    for i in range(nums):
        if DATA_FORMAT in names[i]:
            name = _path + names[i]

            hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
            lr_img = cv2.resize(hr_img, (int(SIZE_X / SCALE), int(SIZE_Y / SCALE)), INTERPOLATION)
            lr_img = cv2.resize(lr_img, (SIZE_X, SIZE_Y), INTERPOLATION)
            
            hr_img_mat = np.asarray(hr_img)
            hr_img_matx = np.array([[hr_img_mat[:,:,0], hr_img_mat[:,:,1], hr_img_mat[:,:,2]]])/255.0
            
            lr_img_mat = np.asarray(lr_img)
            lr_img_matx = np.array([[lr_img_mat[:,:,0], lr_img_mat[:,:,1], lr_img_mat[:,:,2]]])/255.0
                        
            label.append(hr_img_matx)
            data.append(lr_img_matx)
    
    data = np.array(data, dtype=float)
    label = np.array(label, dtype=float)
    return data, label

In [21]:
tick1 = cv2.getTickCount()

In [22]:
data, label = parseData(DATAPATH_TRAIN)
h5DataWrite(data, label, FILENAME_TRAIN)
print(FILENAME_TRAIN + " generated")
data, label = parseData(DATAPATH_VALIDATE)
h5DataWrite(data, label, FILENAME_VALIDATE)
print(FILENAME_VALIDATE + " generated")

yayoi_waifu2x_dataTrain_4.h5 generated
yayoi_waifu2x_dataValidate_4.h5 generated


In [23]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("processed time: " + str(tick) + " ms")

processed time: 476 ms
